In [4]:
import numpy as np
import pandas as pd
import psycopg2
import sys
print(sys.path)
sys.path.append('/home/trang/working/projects/Machine-Learning/data_scrapping')
from constants import twitter

['/home/trang/working/softwares/spark-2.4.5-bin-hadoop2.7/python', '/home/trang/working/softwares/spark-2.4.5-bin-hadoop2.7/python/lib/py4j-0.10.8.1-src.zip', '/home/trang/working/projects/Machine-Learning/data_scrapping/analysis_data/twitter', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/trang/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/trang/.local/lib/python3.6/site-packages/IPython/extensions', '/home/trang/.ipython']


#### Data topic analysis
explore users table:
    - what is strength parts of each users

In [5]:
conn = psycopg2.connect(twitter.db_connection)
cur = conn.cursor()
cur.execute('''select * from users where user_id not in ('1', '2', '3', '4', '5')''')
users = pd.DataFrame(cur.fetchall(), columns = ['user_id', 'user_name', 'screen_name', 'location', 'description', 'url', 'raw_data'])
users.iloc[:10,:]

,user_id,user_name,screen_name,location,description,url,raw_data
0,788898706586275840,Towards Data Science,TDataScience,We're Global 🌏,"A Medium publication sharing concepts, ideas, ...",https://t.co/kt7xo7C48P,"{'id': 788898706586275840, 'url': 'https://t.c..."
1,760888575214977024,Caleb Kaiser,KaiserFrose,"New York, USA",magic @Cortex_Deploy. formerly a Cadillac at @...,None,"{'id': 760888575214977024, 'url': None, 'lang'..."
2,3230388598,Mara Averick,dataandme,"Missoula, MT","tidyverse 🥑 @rstudio, 🏀 hoop head, gnashgab, b...",https://t.co/ZANWJjTExr,"{'id': 3230388598, 'url': 'https://t.co/ZANWJj..."
3,2343198944,Data Science Renee,BecomingDataSci,"Harrisonburg, VA",Director of Data Science at @HelioCampus || Be...,https://t.co/zv9NrlwFSg,"{'id': 2343198944, 'url': 'https://t.co/zv9Nrl..."
4,1408142352,Rachel Thomas,math_rachel,"San Francisco, CA",Director of USF Center for Applied Data Ethics...,https://t.co/LMSqxIYyBX,"{'id': 1408142352, 'url': 'https://t.co/LMSqxI..."
5,393033324,Data Science Central,DataScienceCtrl,"Los Angeles, CA",Co-founded by Vincent Granville and part the D...,http://t.co/gbSIHZcKZR,"{'id': 393033324, 'url': 'http://t.co/gbSIHZcK..."
6,376464264,BigDataDan,BigDataDan,Chicago & Nashville USA,"Big Data and Analytics #Teradata #Hadoop, #Rea...",None,"{'id': 376464264, 'url': None, 'lang': None, '..."
7,216939636,Andrew Ng,AndrewYNg,"Palo Alto, CA",Co-Founder of Coursera; Stanford CS adjunct fa...,https://t.co/XidcMETENd,"{'id': 216939636, 'url': 'https://t.co/XidcMET..."
8,215035672,Caitlin Hudon 👩🏼‍💻,beeonaposy,"Austin, TX","Data, traveling, and tacos. New mom. Lead Data...",https://t.co/JRQNzorJCb,"{'id': 215035672, 'url': 'https://t.co/JRQNzor..."
9,17449590,SmartData Collective,SmartDataCo,"New York, NY",SmartData Collective is one of the largest & t...,https://t.co/69ife4YNnG,"{'id': 17449590, 'url': 'https://t.co/69ife4YN..."


In [6]:
users = pd.DataFrame(cur.fetchall(), columns = ['user_id', 'user_name', 'screen_name', 'location', 'description', 'url', 'raw_data'])
users[['user_id', 'user_name', 'description']]
strengths = pd.Series(('a medium publication', '?', '?', 'data scientist', 'deep learning', 
                       'the DSC community on data science, ml, ai, deep learning, dataviz, hadoop, IoT and BI',
                      'big data and analytics', 'ai, machinelearning, deeplearning, moocs', 'data scientist', 
                      'the community covering techincal about bigdata, ai, bi, cloud, analytics, IoT', 'consultant analyst',
                      'digital technical, data science and ai', 'the DSC community on data science, ml, ai, deep learning, dataviz, hadoop, IoT and BI',
                      'data scientist'))
users = users.assign(strengths=strengths)
users[['user_name', 'strengths']]
print(users.shape)
users.groupby(('strengths')).count()['user_id'].sort_values(ascending=False)

(14, 8)


strengths
the community covering techincal about bigdata, ai, bi, cloud, analytics, IoT            0
the DSC community on data science, ml, ai, deep learning, dataviz, hadoop, IoT and BI    0
digital technical, data science and ai                                                   0
deep learning                                                                            0
data scientist                                                                           0
consultant analyst                                                                       0
big data and analytics                                                                   0
ai, machinelearning, deeplearning, moocs                                                 0
a medium publication                                                                     0
?                                                                                        0
Name: user_id, dtype: int64

#### Questions
??? how to know users dataset is enough to continue analysis Data topic
- Estimated time: 4.2 days (evening 01st - 04th-08-2020)
- evening 01st: What are main sharings of each followers in Data topic?

additional tasks:
- how does Toward Data science works?

In [7]:
cur.execute('select * from tweets;')
tweets = pd.DataFrame(cur.fetchall(), 
                      columns = ['tweet_id', 'text', 'user_mentions', 'raw_data', 'created_date', 'updated_date', 'inserted_date', 'user_id'])
tweets.shape

(1206, 8)

In [8]:
pd.set_option('display.min_rows', 300)
pd.set_option('display.max_rows', 500)
# tweets['user_id'] = tweets['raw_data']['user']['id']
print(tweets.loc[0:1,['text', 'created_date', 'raw_data']])

                                                text        created_date  \
0  How to use Deep Learning for Time Series Forec... 2020-09-22 09:19:01   
1  Ditching the AWS GUI Console by @cinemarob1  h... 2020-09-22 08:19:00   

                                            raw_data  
0  {'id': 1308334979743653888, 'geo': None, 'lang...  
1  {'id': 1308319877267152904, 'geo': None, 'lang...  


In [9]:
tweets.iloc[:10,:]

,tweet_id,text,user_mentions,raw_data,created_date,updated_date,inserted_date,user_id
0,1308334979743653888,How to use Deep Learning for Time Series Forec...,"[{'id': 1194330047693373441, 'name': 'Christop...","{'id': 1308334979743653888, 'geo': None, 'lang...",2020-09-22 09:19:01,2020-09-22 16:32:43.249097,2020-09-22 16:32:43.249097,788898706586275840
1,1308319877267152904,Ditching the AWS GUI Console by @cinemarob1 h...,"[{'id': 400271796, 'name': 'Robert Boscacci', ...","{'id': 1308319877267152904, 'geo': None, 'lang...",2020-09-22 08:19:00,2020-09-22 16:32:43.258021,2020-09-22 16:32:43.258021,788898706586275840
2,1308304022667800576,How to Choose the Right Machine Learning Algor...,"[{'id': 1140864609865256965, 'name': 'Sara A. ...","{'id': 1308304022667800576, 'geo': None, 'lang...",2020-09-22 07:16:00,2020-09-22 16:32:43.260080,2020-09-22 16:32:43.260080,788898706586275840
3,1308294208797642757,Clustering using k-Means with implementation b...,[],"{'id': 1308294208797642757, 'geo': None, 'lang...",2020-09-22 06:37:00,2020-09-22 16:32:43.261965,2020-09-22 16:32:43.261965,788898706586275840
4,1308273823326908419,Why You Shouldn’t Go to Casinos (3 Statistical...,"[{'id': 380448211, 'name': 'Tomi Mester', 'id_...","{'id': 1308273823326908419, 'geo': None, 'lang...",2020-09-22 05:16:00,2020-09-22 16:32:43.264226,2020-09-22 16:32:43.264226,788898706586275840
5,1308267533523464193,A Data Warehouse Implementation on AWS by Anto...,[],"{'id': 1308267533523464193, 'geo': None, 'lang...",2020-09-22 04:51:01,2020-09-22 16:32:43.266271,2020-09-22 16:32:43.266271,788898706586275840
6,1308242871934386177,Linear Regression v.s. Neural Networks by Roma...,[],"{'id': 1308242871934386177, 'geo': None, 'lang...",2020-09-22 03:13:01,2020-09-22 16:32:43.268282,2020-09-22 16:32:43.268282,788898706586275840
7,1308230288837926914,Are You A Senior Data Scientist? by Ori Cohen ...,[],"{'id': 1308230288837926914, 'geo': None, 'lang...",2020-09-22 02:23:01,2020-09-22 16:32:43.270294,2020-09-22 16:32:43.270294,788898706586275840
8,1308217452355088387,Counting Cards With A Neural Net by @tonester5...,"[{'id': 777531696, 'name': 'Tony Yiu', 'id_str...","{'id': 1308217452355088387, 'geo': None, 'lang...",2020-09-22 01:32:00,2020-09-22 16:32:43.272333,2020-09-22 16:32:43.272333,788898706586275840
9,1308197824010563586,"Object-oriented programming is dead. Wait, rea...","[{'id': 952159198686703616, 'name': 'Rhea Mout...","{'id': 1308197824010563586, 'geo': None, 'lang...",2020-09-22 00:14:01,2020-09-22 16:32:43.274538,2020-09-22 16:32:43.274538,788898706586275840


In [15]:
print(users.dtypes)
print(users.shape)
type(tweets.iloc[:0,:]['user_id'])

user_id        object
user_name      object
screen_name    object
location       object
description    object
url            object
raw_data       object
strengths      object
dtype: object
(14, 8)


pandas.core.series.Series

In [11]:
# tweets['user_id'] = tweets['raw_data'].apply(lambda x:str(x['user']['id_str']))
tweet_user_merging = tweets.merge(users, on='user_id', how='inner', suffixes=['_t','_u'])
tweet_user_merging.shape

(0, 15)

In [14]:
tweet_user_merging.iloc[:10,:][['user_id', 'user_name']]

,user_id,user_name


In [ ]:
tweet_user_merging[['text', 'raw_data']]

In [ ]:
tweet_user_merging[['user_id', 'user_name', 'description','text',  'created_date']]


In [ ]:
instered_users = tweet_user_merging['user_id'].isin(['788898706586275840','2343198944', '14174897', '17449590'])
# tweet_user_merging[['user_id','user_name', 'location', 'description', 'text', 'created_date']].loc[instered_users,:]

what are tweets sharings mainly of each followers?
   * Data Science Central: current technologies trending: tools, kind of Deep Learning methods, analysis methods ... real case studies and instructions
   * Data science Renee: explore sharing, improvement sharing (expert level)
   * Toward Data Science: ideas, projects, case study, learning path completely processings
   * Smart Data Collective: applying of data in bussiness 

Find beginer analyst level of followers

What are next steps for analysis, collecting data planning? Data minding?
   - Find more trending, applying and sharing followers or add more type of followers?
       + following more people/branding: add 11 followers
       ! put a question in timeline and understand how it works
       